In [57]:
from tensorflow.keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Conv2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import pickle
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from keras import backend as k
from keras.applications.inception_v3 import InceptionV3
from keras.utils import np_utils
from sklearn import svm


In [2]:
def loadPickleData(file):
    data = pickle.load(open(file, 'rb'));
    images = data[0];
    labels = data[1];
    return [images, labels]

In [3]:
[images,labels] = loadPickleData("./../Data/pickledFiles/Num_SingleDigit_3channel.p")

In [4]:
y = np_utils.to_categorical(labels);
print(y);
print(y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(2000, 10)


In [30]:
X = images
print(X.shape)
# X = np.reshape(X, (2000,139, 139,3));

(2000, 60, 40, 3)


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2);

In [35]:
model = applications.VGG19(weights = "imagenet", include_top=False,\
                           input_shape = (60,40, 3), pooling = 'max')

In [36]:
inputt = Input(shape =  (60,40,3), name = 'captcha_input')

In [37]:
output_vgg16_conv = model(inputt);

In [38]:
model_final = Model(inputt,output_vgg16_conv)

In [39]:
model_final.compile(loss = "categorical_crossentropy",\
                    optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])
print(model_final.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
captcha_input (InputLayer)   [(None, 60, 40, 3)]       0         
_________________________________________________________________
vgg19 (Functional)           (None, 512)               20024384  
Total params: 20,024,384
Trainable params: 20,024,384
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
X_train = model_final.predict(X_train, batch_size = 200, verbose=1)


8/8 [==============================] - 5s 578ms/step


In [41]:
X_test = model_final.predict(X_test, batch_size = 100, verbose=1);

4/4 [==============================] - 1s 299ms/step


In [42]:
print(X_train.shape)

(1600, 512)


In [43]:
resize = np.prod(X_train.shape[1:])

In [44]:
X_train = np.reshape(X_train, (1600, resize));
X_test = np.reshape(X_test, (400, resize));

In [45]:
bf_train_s = X_train
bf_test_s = X_test

In [46]:
y_train_mc = np.argmax(y_train, axis=1)
y_test_mc = np.argmax(y_test, axis = 1)

In [53]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', verbose=1,max_iter=10000)

In [54]:
clf.fit(bf_train_s,y_train_mc);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.8s finished


In [55]:
print(clf.score(bf_test_s, y_test_mc));


0.255


In [58]:
multiClassSVM = svm.LinearSVC(loss = 'squared_hinge', penalty = 'l2',C=1/5000 ,max_iter=800000,verbose=1) 
multiClassSVM.fit(bf_train_s,y_train_mc)

[LibLinear]

LinearSVC(C=0.0002, max_iter=800000, verbose=1)

In [61]:
multiClassSVM.score(bf_test_s, y_test_mc)

0.305